In [ ]:
swiss_file = "swiss_tournament_20251028_170637.pkl"

# Swiss Tournament Analysis

This notebook analyzes the results from a Swiss-system tournament between checkpoints.

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path("../..").resolve()))
import os

if os.path.basename(os.getcwd()) == "E02_win_rate":
    os.chdir("../..")

In [3]:
import pickle
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

## Load Tournament Data

In [4]:
# Load the tournament results
with open(swiss_file, "rb") as f:
    tournament_data = pickle.load(f)

print("Tournament data loaded successfully!")
print(f"\nAvailable keys: {tournament_data.keys()}")

Tournament data loaded successfully!

Available keys: dict_keys(['config', 'final_standings', 'match_results', 'timestamp'])


## Tournament Configuration

In [5]:
config = tournament_data["config"]
print(f"Number of Rounds: {config['num_rounds']}")
print(f"Double Swiss: {config['double_swiss']}")
print(f"McMahon Scoring: {config['mcmahon']}")
print(f"Checkpoint Folder: {config['folder']}")
print(f"Timestamp: {tournament_data['timestamp']}")

Number of Rounds: 5
Double Swiss: True
McMahon Scoring: True
Checkpoint Folder: CHECKPOINTS//E02_win_rate//
Timestamp: 20251028_170637


## Final Standings

In [6]:
# Convert standings to DataFrame
standings = tournament_data["final_standings"]
standings_df = pd.DataFrame(standings)

# Display top 20
print(f"\nTotal participants: {len(standings_df)}")
print(f"\nTop 20 Final Standings:")
top_20 = standings_df.head(20)[
    ["name", "score", "wins", "draws", "losses", "matches_played", "epoch"]
]
print(top_20.to_string(index=False))

standings_df.head()


Total participants: 1043

Top 20 Final Standings:
                                 name  score  wins  draws  losses  matches_played  epoch
20251023_1438-E02_win_rate_epoch_1011    9.0     9      0       1              10   1011
20251023_1435-E02_win_rate_epoch_0986    9.0     9      0       1              10    986
20251023_1428-E02_win_rate_epoch_0910    9.0     9      0       1              10    910
20251023_1422-E02_win_rate_epoch_0828    9.0     9      0       1              10    828
20251023_1421-E02_win_rate_epoch_0809    9.0     9      0       1              10    809
20251023_1414-E02_win_rate_epoch_0709    9.0     9      0       1              10    709
20251023_1410-E02_win_rate_epoch_0657    9.0     9      0       1              10    657
20251023_1405-E02_win_rate_epoch_0586    9.0     9      0       1              10    586
20251023_1401-E02_win_rate_epoch_0534    9.0     9      0       1              10    534
20251023_1350-E02_win_rate_epoch_0378    9.0     9      0  

,name,path,epoch,score,wins,draws,losses,matches_played
0,20251023_1438-E02_win_rate_epoch_1011,CHECKPOINTS\E02_win_rate\20251023_1438-E02_win...,1011,9.0,9,0,1,10
1,20251023_1435-E02_win_rate_epoch_0986,CHECKPOINTS\E02_win_rate\20251023_1435-E02_win...,986,9.0,9,0,1,10
2,20251023_1428-E02_win_rate_epoch_0910,CHECKPOINTS\E02_win_rate\20251023_1428-E02_win...,910,9.0,9,0,1,10
3,20251023_1422-E02_win_rate_epoch_0828,CHECKPOINTS\E02_win_rate\20251023_1422-E02_win...,828,9.0,9,0,1,10
4,20251023_1421-E02_win_rate_epoch_0809,CHECKPOINTS\E02_win_rate\20251023_1421-E02_win...,809,9.0,9,0,1,10


## Score Distribution

In [7]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=standings_df["score"],
        nbinsx=30,
        name="Score Distribution",
        marker_color="blue",
    )
)

fig.update_layout(
    title="Score Distribution Across All Participants",
    xaxis_title="Total Score",
    yaxis_title="Number of Bots",
    height=400,
)
fig.show()

## Epoch vs Performance

In [8]:
# Scatter plot: Epoch vs Score
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_df["epoch"],
        y=standings_df["score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_df["score"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Score"),
        ),
        text=standings_df["name"].str[-15:],  # Show last 15 chars of name
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>Score: %{y}<extra></extra>",
    )
)

fig.update_layout(
    title="Epoch vs Tournament Score",
    xaxis_title="Training Epoch",
    yaxis_title="Tournament Score",
    height=500,
)
fig.show()

In [9]:
# Win rate vs epoch
standings_df["win_rate"] = standings_df["wins"] / standings_df["matches_played"]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_df["epoch"],
        y=standings_df["win_rate"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_df["win_rate"],
            colorscale="RdYlGn",
            showscale=True,
            colorbar=dict(title="Win Rate"),
        ),
        text=standings_df["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>Win Rate: %{y:.3f}<extra></extra>",
    )
)

fig.update_layout(
    title="Win Rate vs Training Epoch",
    xaxis_title="Training Epoch",
    yaxis_title="Win Rate",
    height=500,
)
fig.show()

## Match Results Analysis

In [10]:
# Convert match results to DataFrame
matches = tournament_data["match_results"]
matches_df = pd.DataFrame(matches)

print(f"Total matches played: {len(matches_df)}")
print(f"\nMatch results columns: {matches_df.columns.tolist()}")
matches_df.head(10)

Total matches played: 5210

Match results columns: ['round', 'match_id', 'player1', 'player2', 'player1_epoch', 'player2_epoch', 'p1_wins', 'p2_wins', 'draws']


,round,match_id,player1,player2,player1_epoch,player2_epoch,p1_wins,p2_wins,draws
0,1,R1_M1_G1,20251023_1439-E02_win_rate_epoch_1014,20251023_1439-E02_win_rate_epoch_1013,1014,1013,1,0,0
1,1,R1_M1_G2,20251023_1439-E02_win_rate_epoch_1013,20251023_1439-E02_win_rate_epoch_1014,1013,1014,1,0,0
2,1,R1_M2_G1,20251023_1438-E02_win_rate_epoch_1012,20251023_1438-E02_win_rate_epoch_1011,1012,1011,0,1,0
3,1,R1_M2_G2,20251023_1438-E02_win_rate_epoch_1011,20251023_1438-E02_win_rate_epoch_1012,1011,1012,1,0,0
4,1,R1_M3_G1,20251023_1438-E02_win_rate_epoch_1010,20251023_1438-E02_win_rate_epoch_1009,1010,1009,0,1,0
5,1,R1_M3_G2,20251023_1438-E02_win_rate_epoch_1009,20251023_1438-E02_win_rate_epoch_1010,1009,1010,0,1,0
6,1,R1_M4_G1,20251023_1438-E02_win_rate_epoch_1008,20251023_1438-E02_win_rate_epoch_1007,1008,1007,0,1,0
7,1,R1_M4_G2,20251023_1438-E02_win_rate_epoch_1007,20251023_1438-E02_win_rate_epoch_1008,1007,1008,0,1,0
8,1,R1_M5_G1,20251023_1438-E02_win_rate_epoch_1006,20251023_1438-E02_win_rate_epoch_1005,1006,1005,1,0,0
9,1,R1_M5_G2,20251023_1438-E02_win_rate_epoch_1005,20251023_1438-E02_win_rate_epoch_1006,1005,1006,1,0,0


In [11]:
# Analyze match outcomes
total_p1_wins = matches_df["p1_wins"].sum()
total_p2_wins = matches_df["p2_wins"].sum()
total_draws = matches_df["draws"].sum()
total_games = total_p1_wins + total_p2_wins + total_draws

print("=" * 60)
print("OVERALL MATCH STATISTICS")
print("=" * 60)
print(f"Player 1 Wins: {total_p1_wins} ({total_p1_wins/total_games*100:.1f}%)")
print(f"Player 2 Wins: {total_p2_wins} ({total_p2_wins/total_games*100:.1f}%)")
print(f"Draws: {total_draws} ({total_draws/total_games*100:.1f}%)")
print(f"Total Games: {total_games}")
print("=" * 60)

OVERALL MATCH STATISTICS
Player 1 Wins: 2579 (49.5%)
Player 2 Wins: 2612 (50.1%)
Draws: 19 (0.4%)
Total Games: 5210


In [12]:
# Match outcomes pie chart
fig = go.Figure(
    data=[
        go.Pie(
            labels=["Player 1 Wins", "Player 2 Wins", "Draws"],
            values=[total_p1_wins, total_p2_wins, total_draws],
            marker_colors=["#1f77b4", "#ff7f0e", "#2ca02c"],
        )
    ]
)

fig.update_layout(title="Overall Match Outcomes Distribution", height=400)
fig.show()

## Performance by Round

In [13]:
# Analyze performance by round
round_stats = matches_df.groupby("round").agg(
    {
        "p1_wins": "sum",
        "p2_wins": "sum",
        "draws": "sum",
    }
)

round_stats["total_games"] = (
    round_stats["p1_wins"] + round_stats["p2_wins"] + round_stats["draws"]
)
round_stats["p1_win_rate"] = round_stats["p1_wins"] / round_stats["total_games"]
round_stats["draw_rate"] = round_stats["draws"] / round_stats["total_games"]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Games per Round", "Outcome Rates"))

# Total games per round
fig.add_trace(
    go.Scatter(
        x=round_stats.index,
        y=round_stats["total_games"],
        mode="lines+markers",
        name="Total Games",
    ),
    row=1,
    col=1,
)

# Outcome rates
fig.add_trace(
    go.Scatter(
        x=round_stats.index,
        y=round_stats["p1_win_rate"],
        mode="lines+markers",
        name="P1 Win Rate",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=round_stats.index,
        y=round_stats["draw_rate"],
        mode="lines+markers",
        name="Draw Rate",
    ),
    row=2,
    col=1,
)

fig.update_xaxes(title_text="Round", row=2, col=1)
fig.update_yaxes(title_text="Games", row=1, col=1)
fig.update_yaxes(title_text="Rate", row=2, col=1)
fig.update_layout(height=700, title_text="Round-by-Round Analysis")
fig.show()

## Top Performers Analysis

In [14]:
# Top 10 performers detailed stats
top_10 = standings_df.head(10).copy()
top_10["win_rate"] = top_10["wins"] / top_10["matches_played"]
top_10["draw_rate"] = top_10["draws"] / top_10["matches_played"]
top_10["loss_rate"] = top_10["losses"] / top_10["matches_played"]

fig = go.Figure()

# Stacked bar chart
fig.add_trace(
    go.Bar(
        name="Wins",
        x=top_10["name"].str[-15:],
        y=top_10["win_rate"],
        marker_color="green",
    )
)

fig.add_trace(
    go.Bar(
        name="Draws",
        x=top_10["name"].str[-15:],
        y=top_10["draw_rate"],
        marker_color="yellow",
    )
)

fig.add_trace(
    go.Bar(
        name="Losses",
        x=top_10["name"].str[-15:],
        y=top_10["loss_rate"],
        marker_color="red",
    )
)

fig.update_layout(
    barmode="stack",
    title="Top 10 Performers - Match Outcome Distribution",
    xaxis_title="Bot Name (last 15 chars)",
    yaxis_title="Rate",
    height=500,
)
fig.show()

## Epoch Correlation Analysis

In [15]:
from scipy import stats

# Calculate correlation between epoch and performance metrics
corr_score, p_score = stats.pearsonr(standings_df["epoch"], standings_df["score"])
corr_winrate, p_winrate = stats.pearsonr(
    standings_df["epoch"], standings_df["win_rate"]
)

print("=" * 60)
print("EPOCH CORRELATION ANALYSIS")
print("=" * 60)
print(f"Epoch vs Score:")
print(f"  Correlation: {corr_score:.4f}")
print(f"  P-value: {p_score:.4e}")
print(f"  Significant: {'Yes' if p_score < 0.05 else 'No'}")
print()
print(f"Epoch vs Win Rate:")
print(f"  Correlation: {corr_winrate:.4f}")
print(f"  P-value: {p_winrate:.4e}")
print(f"  Significant: {'Yes' if p_winrate < 0.05 else 'No'}")
print("=" * 60)

EPOCH CORRELATION ANALYSIS
Epoch vs Score:
  Correlation: 0.0112
  P-value: 7.1862e-01
  Significant: No

Epoch vs Win Rate:
  Correlation: 0.0159
  P-value: 6.0798e-01
  Significant: No


## Summary Statistics

In [16]:
summary = {
    "Total Bots": len(standings_df),
    "Total Rounds": config["num_rounds"],
    "Total Matches": len(matches_df),
    "Avg Score": standings_df["score"].mean(),
    "Median Score": standings_df["score"].median(),
    "Avg Win Rate": standings_df["win_rate"].mean(),
    "Top Score": standings_df["score"].max(),
    "Top Bot": standings_df.iloc[0]["name"][-30:],
    "Top Epoch": standings_df.iloc[0]["epoch"],
}

print("=" * 60)
print("TOURNAMENT SUMMARY")
print("=" * 60)
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key:.<40} {value:.3f}")
    else:
        print(f"{key:.<40} {value}")
print("=" * 60)

TOURNAMENT SUMMARY
Total Bots.............................. 1043
Total Rounds............................ 5
Total Matches........................... 5210
Avg Score............................... 5.000
Median Score............................ 5.000
Avg Win Rate............................ 0.498
Top Score............................... 9.000
Top Bot................................. 3_1438-E02_win_rate_epoch_1011
Top Epoch............................... 1011


## Bradley-Terry Scoring

Calculate Bradley-Terry scores to estimate relative strength of each bot based on match results.

In [17]:
from utils.metrics.bradley_terry import calculate_BradleyTerry

# Build win matrix from match results
# W[i,j] = number of wins of bot i over bot j
bot_names = standings_df["name"].tolist()
n_bots = len(bot_names)

# Create name to index mapping
name_to_idx = {name: idx for idx, name in enumerate(bot_names)}

# Initialize win matrix as DataFrame
W = pd.DataFrame(0.0, index=bot_names, columns=bot_names)

# Populate win matrix from match results
for _, match in matches_df.iterrows():
    p1_name = match["player1"]
    p2_name = match["player2"]

    # Add wins to matrix
    W.loc[p1_name, p2_name] += match["p1_wins"]
    W.loc[p2_name, p1_name] += match["p2_wins"]

    # Handle draws (split equally)
    if match["draws"] > 0:
        W.loc[p1_name, p2_name] += match["draws"] * 0.5
        W.loc[p2_name, p1_name] += match["draws"] * 0.5

print(f"Win matrix constructed: {W.shape}")
print(
    f"Total games recorded: {W.sum().sum() / 2:.0f}"
)  # Divide by 2 since matrix counts each game twice

Win matrix constructed: (1043, 1043)
Total games recorded: 2605


,20251023_1438-E02_win_rate_epoch_1011,20251023_1435-E02_win_rate_epoch_0986,20251023_1428-E02_win_rate_epoch_0910,20251023_1422-E02_win_rate_epoch_0828,20251023_1421-E02_win_rate_epoch_0809,20251023_1414-E02_win_rate_epoch_0709,20251023_1410-E02_win_rate_epoch_0657,20251023_1405-E02_win_rate_epoch_0586,20251023_1401-E02_win_rate_epoch_0534,20251023_1350-E02_win_rate_epoch_0378,...,20251023_1306-E02_win_rate_epoch_0009,20251023_1432-E02_win_rate_epoch_0953,20251023_1429-E02_win_rate_epoch_0927,20251023_1418-E02_win_rate_epoch_0766,20251023_1412-E02_win_rate_epoch_0690,20251023_1404-E02_win_rate_epoch_0579,20251023_1359-E02_win_rate_epoch_0496,20251023_1354-E02_win_rate_epoch_0431,20251023_1336-E02_win_rate_epoch_0171,20251023_1307-E02_win_rate_epoch_0010
20251023_1438-E02_win_rate_epoch_1011,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1435-E02_win_rate_epoch_0986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1428-E02_win_rate_epoch_0910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1422-E02_win_rate_epoch_0828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1421-E02_win_rate_epoch_0809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20251023_1404-E02_win_rate_epoch_0579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1359-E02_win_rate_epoch_0496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1354-E02_win_rate_epoch_0431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20251023_1336-E02_win_rate_epoch_0171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Initialize scores (uniform initialization)
initial_scores = {name: 1.0 for name in bot_names}

# Calculate Bradley-Terry scores
bt_scores = calculate_BradleyTerry(
    score=initial_scores,
    W=W,
    EPOCHS=4,
    diff_threshold=1e-6,
    normalize=False,
    verbose=True,
)

print("\nBradley-Terry calculation complete!")

Calculating Bradley-Terry for 1043 agents.



BT Epoch: 100%|██████████| 1043/1043 [00:11<00:00, 89.63it/s]


Diff norm2: 0.189624
['9.000', '9.000', '9.000', '9.000', '9.000', '9.000', '49.000', '9.000', '9.000', '9.000', '49.000', '9.000', '49.000', '9.000', '4.000', '20.000', '4.000', '4.000', '4.000', '4.000', '4.000', '7.333', '4.000', '7.065', '4.000', '4.000', '4.000', '6.143', '4.000', '4.000', '6.143', '4.000', '100.000', '4.000', '4.000', '4.000', '6.143', '100.000', '4.000', '20.000', '4.000', '42.000', '4.000', '4.000', '4.000', '4.000', '6.143', '6.143', '4.000', '4.000', '3.000', '3.000', '2.333', '2.846', '2.333', '2.333', '2.333', '5.000', '2.333', '2.333', '2.846', '2.333', '4.814', '3.266', '5.355', '3.051', '5.000', '3.010', '3.002', '6.694', '2.333', '2.333', '2.333', '5.000', '2.846', '2.333', '2.333', '2.968', '5.000', '2.333', '3.545', '2.994', '2.333', '3.167', '3.032', '13.000', '2.333', '2.333', '2.333', '7.400', '2.333', '2.333', '2.333', '10.882', '2.333', '2.846', '2.333', '2.333', '6.481', '4.561', '3.410', '2.333', '2.333', '5.571', '2.333', '2.333', '3.340', '5.

BT Epoch: 100%|██████████| 1043/1043 [00:11<00:00, 88.45it/s]


Diff norm2: 1.042125
['175.281', '26.649', '30.362', '48.079', '47.663', '50.507', '408.428', '151.222', '67.163', '24.904', '714.515', '165.365', '262.657', '24.581', '5.502', '93.018', '18.884', '9.937', '10.285', '4.610', '10.317', '32.669', '16.300', '25.952', '9.300', '13.315', '7.308', '19.559', '8.121', '7.860', '9.693', '7.083', '783.182', '7.041', '19.614', '8.512', '16.844', '58.010', '8.554', '41.234', '8.637', '20.155', '10.109', '7.518', '10.497', '12.421', '17.853', '9.861', '5.116', '8.130', '9.670', '5.664', '4.213', '5.394', '2.453', '4.055', '3.072', '8.170', '4.207', '3.021', '5.932', '3.862', '11.987', '10.720', '9.242', '6.628', '5.253', '5.159', '4.622', '14.645', '5.158', '3.032', '4.437', '11.523', '3.538', '3.756', '2.870', '5.289', '12.200', '3.219', '5.740', '6.976', '2.595', '8.346', '4.432', '37.940', '2.365', '2.363', '6.011', '12.031', '3.670', '8.188', '5.800', '20.599', '3.394', '4.733', '5.092', '3.679', '18.395', '5.841', '6.744', '5.276', '3.800', '1

BT Epoch: 100%|██████████| 1043/1043 [00:11<00:00, 90.00it/s]


Diff norm2: 2.699372
['852.648', '47.972', '47.711', '100.478', '129.183', '146.471', '1212.833', '1019.884', '183.928', '32.872', '3111.036', '392.157', '490.746', '41.663', '7.008', '179.275', '31.510', '17.408', '18.084', '3.990', '16.459', '87.609', '33.803', '58.199', '15.605', '25.968', '9.770', '36.395', '9.680', '9.558', '11.570', '9.386', '842.524', '9.122', '42.454', '12.971', '33.521', '462.369', '11.871', '39.482', '15.946', '27.890', '18.262', '12.403', '16.613', '23.096', '25.904', '12.275', '5.444', '10.673', '16.487', '7.538', '5.672', '7.099', '2.422', '4.663', '4.285', '11.225', '5.986', '2.837', '8.360', '4.051', '21.262', '18.824', '12.026', '8.971', '6.136', '6.732', '6.200', '21.622', '5.930', '3.493', '6.920', '14.693', '4.259', '4.253', '3.222', '6.766', '18.966', '3.937', '7.779', '8.766', '2.535', '13.321', '4.987', '62.777', '2.428', '1.950', '10.659', '18.353', '3.691', '18.043', '7.022', '32.370', '4.117', '5.801', '7.302', '3.869', '25.413', '4.995', '9.99

BT Epoch: 100%|██████████| 1043/1043 [00:11<00:00, 90.23it/s]


Diff norm2: 5.600368
['1968.992', '59.461', '68.674', '137.677', '224.153', '290.511', '2398.195', '3466.911', '322.881', '34.941', '7986.679', '628.471', '744.488', '53.637', '8.111', '242.051', '41.475', '23.587', '26.183', '3.575', '21.048', '159.464', '56.370', '99.653', '19.260', '39.817', '10.568', '51.487', '9.803', '10.359', '12.225', '10.834', '2035.412', '10.199', '70.443', '16.538', '51.361', '179.632', '14.362', '53.427', '23.075', '29.107', '25.507', '13.768', '20.604', '31.469', '32.718', '13.460', '5.680', '11.551', '19.837', '8.827', '6.972', '8.007', '2.456', '5.187', '5.083', '14.109', '7.241', '2.717', '10.832', '3.668', '29.169', '24.294', '12.989', '10.175', '6.857', '7.941', '7.296', '26.701', '5.991', '3.804', '8.764', '15.638', '4.785', '4.507', '3.460', '7.228', '22.956', '4.651', '9.751', '9.678', '2.412', '15.826', '5.282', '87.670', '2.455', '1.725', '14.372', '23.644', '3.361', '30.191', '7.188', '46.101', '4.650', '6.250', '8.101', '3.622', '27.728', '4.52

## Bradley-Terry Rankings

In [22]:
# Create Bradley-Terry rankings DataFrame
bt_df = pd.DataFrame(
    [
        {
            "name": name,
            "bt_score": score,
            "epoch": standings_df[standings_df["name"] == name]["epoch"].iloc[0],
        }
        for name, score in bt_scores.items()
    ]
)

bt_df = bt_df.sort_values("bt_score", ascending=False).reset_index(drop=True)
bt_df["bt_rank"] = range(1, len(bt_df) + 1)

# Merge with original standings
standings_with_bt = standings_df.merge(
    bt_df[["name", "bt_score", "bt_rank"]], on="name", how="left"
)

# Compare Swiss rank vs BT rank
standings_with_bt["rank_diff"] = (
    standings_with_bt.index + 1 - standings_with_bt["bt_rank"]
)

print("Top 20 by Bradley-Terry Score:")
print(
    standings_with_bt.head(20)[
        ["name", "score", "bt_score", "bt_rank", "rank_diff", "epoch"]
    ].to_string(index=False)
)

standings_with_bt.head()

Top 20 by Bradley-Terry Score:
                                 name  score    bt_score  bt_rank  rank_diff  epoch
20251023_1438-E02_win_rate_epoch_1011    9.0 1968.991610        5         -4   1011
20251023_1435-E02_win_rate_epoch_0986    9.0   59.461143       22        -20    986
20251023_1428-E02_win_rate_epoch_0910    9.0   68.674244       20        -17    910
20251023_1422-E02_win_rate_epoch_0828    9.0  137.677081       15        -11    828
20251023_1421-E02_win_rate_epoch_0809    9.0  224.153344       12         -7    809
20251023_1414-E02_win_rate_epoch_0709    9.0  290.511369        9         -3    709
20251023_1410-E02_win_rate_epoch_0657    9.0 2398.195368        3          4    657
20251023_1405-E02_win_rate_epoch_0586    9.0 3466.910825        2          6    586
20251023_1401-E02_win_rate_epoch_0534    9.0  322.880859        8          1    534
20251023_1350-E02_win_rate_epoch_0378    9.0   34.940885       33        -23    378
20251023_1342-E02_win_rate_epoch_0260    9.0 

,name,path,epoch,score,wins,draws,losses,matches_played,win_rate,bt_score,bt_rank,rank_diff
0,20251023_1438-E02_win_rate_epoch_1011,CHECKPOINTS\E02_win_rate\20251023_1438-E02_win...,1011,9.0,9,0,1,10,0.9,1968.991610,5,-4
1,20251023_1435-E02_win_rate_epoch_0986,CHECKPOINTS\E02_win_rate\20251023_1435-E02_win...,986,9.0,9,0,1,10,0.9,59.461143,22,-20
2,20251023_1428-E02_win_rate_epoch_0910,CHECKPOINTS\E02_win_rate\20251023_1428-E02_win...,910,9.0,9,0,1,10,0.9,68.674244,20,-17
3,20251023_1422-E02_win_rate_epoch_0828,CHECKPOINTS\E02_win_rate\20251023_1422-E02_win...,828,9.0,9,0,1,10,0.9,137.677081,15,-11
4,20251023_1421-E02_win_rate_epoch_0809,CHECKPOINTS\E02_win_rate\20251023_1421-E02_win...,809,9.0,9,0,1,10,0.9,224.153344,12,-7


In [23]:
# Bradley-Terry score distribution
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=bt_df["bt_score"],
        nbinsx=30,
        name="BT Score Distribution",
        marker_color="purple",
    )
)

fig.update_layout(
    title="Bradley-Terry Score Distribution",
    xaxis_title="Bradley-Terry Score",
    yaxis_title="Number of Bots",
    height=400,
)
fig.show()

## Bradley-Terry vs Swiss Score Comparison

In [24]:
# Scatter plot: Swiss score vs BT score
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["score"],
        y=standings_with_bt["bt_score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["epoch"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Epoch"),
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Swiss: %{x:.2f}<br>BT: %{y:.4f}<br>Epoch: %{marker.color}<extra></extra>",
    )
)

# Add diagonal line (perfect correlation)
min_val = 0
max_swiss = standings_with_bt["score"].max()
max_bt = standings_with_bt["bt_score"].max()

fig.add_trace(
    go.Scatter(
        x=[0, max_swiss],
        y=[0, max_bt * (max_swiss / max_swiss)],
        mode="lines",
        line=dict(dash="dash", color="gray"),
        name="Reference",
        showlegend=False,
    )
)

fig.update_layout(
    title="Swiss Score vs Bradley-Terry Score",
    xaxis_title="Swiss Tournament Score",
    yaxis_title="Bradley-Terry Score",
    height=500,
)
fig.show()

In [25]:
# Correlation analysis
from scipy.stats import pearsonr, spearmanr

pearson_corr, pearson_p = pearsonr(
    standings_with_bt["score"], standings_with_bt["bt_score"]
)
spearman_corr, spearman_p = spearmanr(
    standings_with_bt["score"], standings_with_bt["bt_score"]
)

print("=" * 60)
print("CORRELATION ANALYSIS: Swiss Score vs Bradley-Terry Score")
print("=" * 60)
print(f"Pearson Correlation: {pearson_corr:.4f} (p={pearson_p:.4e})")
print(f"Spearman Correlation: {spearman_corr:.4f} (p={spearman_p:.4e})")
print("=" * 60)

CORRELATION ANALYSIS: Swiss Score vs Bradley-Terry Score
Pearson Correlation: 0.1833 (p=2.4708e-09)
Spearman Correlation: 0.8772 (p=0.0000e+00)


## Bradley-Terry Score vs Epoch

In [26]:
# BT score vs epoch
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["epoch"],
        y=standings_with_bt["bt_score"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["bt_score"],
            colorscale="RdYlGn",
            showscale=True,
            colorbar=dict(title="BT Score"),
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Epoch: %{x}<br>BT Score: %{y:.4f}<extra></extra>",
    )
)

fig.update_layout(
    title="Bradley-Terry Score vs Training Epoch",
    xaxis_title="Training Epoch",
    yaxis_title="Bradley-Terry Score",
    height=500,
)
fig.show()

In [27]:
# Epoch correlation with BT score
bt_epoch_corr, bt_epoch_p = pearsonr(
    standings_with_bt["epoch"], standings_with_bt["bt_score"]
)

print("=" * 60)
print("EPOCH CORRELATION WITH BRADLEY-TERRY")
print("=" * 60)
print(f"Correlation: {bt_epoch_corr:.4f}")
print(f"P-value: {bt_epoch_p:.4e}")
print(f"Significant: {'Yes' if bt_epoch_p < 0.05 else 'No'}")
print("=" * 60)

EPOCH CORRELATION WITH BRADLEY-TERRY
Correlation: -0.0049
P-value: 8.7557e-01
Significant: No


## Ranking Disagreements

Identify bots where Swiss ranking and Bradley-Terry ranking differ significantly.

In [28]:
# Find largest ranking disagreements
standings_with_bt["swiss_rank"] = standings_with_bt.index + 1
standings_with_bt["abs_rank_diff"] = abs(standings_with_bt["rank_diff"])

disagreements = standings_with_bt.nlargest(10, "abs_rank_diff")[
    ["name", "swiss_rank", "bt_rank", "rank_diff", "score", "bt_score", "epoch"]
]

print("Top 10 Ranking Disagreements:")
print(disagreements.to_string(index=False))

Top 10 Ranking Disagreements:
                                 name  swiss_rank  bt_rank  rank_diff  score  bt_score  epoch
20251023_1434-E02_win_rate_epoch_0974         182      746       -564    6.0  0.774439    974
20251023_1308-E02_win_rate_epoch_0014         853      298        555    4.0  3.338343     14
20251023_1405-E02_win_rate_epoch_0588         265      773       -508    6.0  0.684291    588
20251023_1405-E02_win_rate_epoch_0594         725      235        490    4.0  4.293868    594
20251023_1336-E02_win_rate_epoch_0174         817      331        486    4.0  3.002386    174
20251023_1325-E02_win_rate_epoch_0023         850      394        456    4.0  2.417409     23
20251023_1325-E02_win_rate_epoch_0014         852      408        444    4.0  2.345837     14
20251023_1333-E02_win_rate_epoch_0127         151      588       -437    7.0  1.350460    127
20251023_1339-E02_win_rate_epoch_0217         341      778       -437    6.0  0.669977    217
20251023_1336-E02_win_rate_epo

In [29]:
# Visualize ranking differences
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=standings_with_bt["swiss_rank"],
        y=standings_with_bt["bt_rank"],
        mode="markers",
        marker=dict(
            size=8,
            color=standings_with_bt["abs_rank_diff"],
            colorscale="Reds",
            showscale=True,
            colorbar=dict(title="Rank Difference"),
        ),
        text=standings_with_bt["name"].str[-15:],
        hovertemplate="<b>%{text}</b><br>Swiss Rank: %{x}<br>BT Rank: %{y}<br>Diff: %{marker.color}<extra></extra>",
    )
)

# Add diagonal (perfect agreement)
max_rank = len(standings_with_bt)
fig.add_trace(
    go.Scatter(
        x=[1, max_rank],
        y=[1, max_rank],
        mode="lines",
        line=dict(dash="dash", color="gray"),
        name="Perfect Agreement",
        showlegend=False,
    )
)

fig.update_layout(
    title="Swiss Rank vs Bradley-Terry Rank",
    xaxis_title="Swiss Rank",
    yaxis_title="Bradley-Terry Rank",
    height=500,
)
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(autorange="reversed")
fig.show()